In [69]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd

plt.ion()   # interactive mode

In [70]:
import pickle

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# Keras libraries for CNN
from keras.models import Sequential

In [71]:
train_imgs = pickle.load(open("train_images_512.pk",'rb'), encoding='bytes')
train_labels = pickle.load(open("train_labels_512.pk",'rb'), encoding='bytes')
test_imgs = pickle.load(open("test_images_512.pk",'rb'), encoding='bytes')

In [65]:
# def load_pk(filename):
#     with open(filename, 'rb') as f:
#         return pickle.load(f, encoding='bytes')

# def load_X(filename):
#     X = load_pk(filename)
#     assert X.size(2) == X.size(3) == SIZE['raw']
#     return SCALE * (X[:, :, :, :] + SHIFT) #got rid of the :1 here on the second axis, transformations wanted to see all 3 values


# def load_y(filename):
#     y = load_pk(filename)
#     return y.type(torch.long)

In [66]:
# train_imgs = load_X("train_images_512.pk")
# train_labels = load_y("train_labels_512.pk")
# test_imgs = load_X("test_images_512.pk")

In [72]:
# ==================== 2. Define constants ====================

# Verbose levels
VERBOSE = {
    'progress': True,  # Prints progress along the way
    'augmenting': False,  # Notify whenever an image transformation goes wrong
    'verify': False,  # Shows an image of representative transformed images
}

# Pathnames and filenames
DIR = {
    'data': os.path.join('..', 'data'),
    'images': os.path.join('..', 'images')
}
FILENAMES = {
    'X_train': os.path.join(DIR['data'], 'train_images_512.pk'),
    'y_train': os.path.join(DIR['data'], 'train_labels_512.pk'),
    'X_test': os.path.join(DIR['data'], 'test_images_512.pk'),
}

# Parameters to load raw data
SHIFT, SCALE = 1, 127.5  # data = (data + 1) * 127.5
SIZE = {'raw': 512, 'final': 256}  # size of raw data and data after downsampling

# Relative amount of data used for training vs validation
TRAIN_VALID_RATIO = 2 / 1  # i.e. ratio of training data : validation data = 2:1

# Parameters for data augmentation
N_TRAIN = 1000  # Train using n = 1000
POS_NEG_RATIO = 1 / 1  # positive:negative samples in augmented training set = 1:1
# (may be backwards)

# Parameters for data augmentation
TRANSFORMATIONS = torchvision.transforms.Compose([
    # Horizontal flip with probability 50%
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    # Rotate by up to 15 degrees either way
    # Translate by ± 2% of image size
    # Expand by up to 15%
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.02, 0.02), scale=(1, 1.15)),  # rotation +
    # Adjust contrast and brightness by up to 40% and 30% respectively
    torchvision.transforms.ColorJitter(contrast=0.4, brightness=0.3),
    # Should randomly add black boxes, but doesn't seem to work
    torchvision.transforms.RandomErasing(p=0.3, scale=(0.01, 0.03), ratio=(1 / 3, 3)),
    # Convert back to torch.tensor
    torchvision.transforms.ToTensor(),
])

In [83]:
# Function to augment training data
def augment(X_in, y_in, transform_fn):
    # Helper to transform a specific image
    # Continues to try for each image until it is successful (takes 1-3 tries)
    def transform(im_):
        while True:
            try:
                return transform_fn(im_)
            except AttributeError:
                verbose_print(f'Something went wrong', VERBOSE['augmenting'])
                continue

    #X_out, y_out = torch.zeros(N_TRAIN, 1, SIZE['final'], SIZE['final']), torch.zeros(N_TRAIN)
    X_out, y_out = torch.zeros(N_TRAIN, 3, SIZE['raw'], SIZE['raw']), torch.zeros(N_TRAIN) #change to work with my data formatting
    f = POS_NEG_RATIO / (1 + POS_NEG_RATIO)
    for i in range(N_TRAIN):
        label = int(torch.rand(1, 1) > f)
        X = X_in[y_in == label]
        print(X.size)
        print(X.shape)
        choice = int(torch.randint(0, X.shape[0], (1, 1))) #had to change to X.shape[0] instead of X.size(0)
        print(X[choice])
        im = torchvision.transforms.functional.to_pil_image(X[choice], mode = 'RGB') #had to specify a mode earlier...
        print(im)
        X_out[i], y_out[i] = transform(im), label
    return X_out, y_out

# Function to visualize transformed training data
def display(X, h=4, w=5):
    s = SIZE['final']
    im = np.empty((1, h * s, w * s))
    idx = np.random.choice(X.size(0), (h, w), replace=False)
    for i in range(h):
        for j in range(w):
            im[:, s * i:s * (i + 1), s * j:s * (j + 1)] = np.array(X[idx[i, j]])
    plt.imshow(im.transpose((1, 2, 0)).repeat(3, 2))
    plt.title('Augmented training data sample')
    plt.axis('off')
    plt.show()

In [74]:
np.array(train_imgs).shape

X_messy = np.array(train_imgs)
X = np.zeros((70,512*512))
X2d = np.zeros((70,512,512))
for i in range(70):
    for j in range(512):
        for k in range(512):
            X[i,j*512+k] = X_messy[i,1,j,k]
            X2d[i,j,k] = X_messy[i,1,j,k]
print(X.shape)
X = (X + 1) * 127.5
X2d = (X + 1) * 127.5
#print(X[:,7])
        
y = np.array(train_labels)

(70, 262144)


In [75]:
toShuffle = np.concatenate((X,y[:,np.newaxis]), axis=1)
np.random.shuffle(toShuffle)
shuffled_y = toShuffle[:,-1] 
shuffled_X = toShuffle[:, :-1]
print(shuffled_X)

[[0.01568556 0.01960695 0.01568556 ... 0.49803928 0.50588205 0.49803928]
 [0.21045551 0.21045551 0.21045551 ... 0.60392439 0.58038846 0.5647029 ]
 [0.08234918 0.         0.00392139 ... 0.49411789 0.62745273 0.87058648]
 ...
 [0.50980344 0.51372483 0.51764622 ... 0.89411482 0.88627204 0.88627204]
 [0.57646707 0.57058498 0.5647029  ... 0.70293948 0.69999844 0.69411635]
 [0.38529173 0.39411485 0.40587902 ... 0.25588587 0.25588587 0.25588587]]


In [76]:
from sklearn.ensemble import RandomForestClassifier

X = shuffled_X
y = shuffled_y

# Parameters for data augmentation
TRANSFORMATIONS = torchvision.transforms.Compose([
    # Horizontal flip with probability 50%
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    # Rotate by up to 15 degrees either way
    # Translate by ± 2% of image size
    # Expand by up to 15%
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.02, 0.02), scale=(1, 1.15)),  # rotation +
    # Adjust contrast and brightness by up to 40% and 30% respectively
    torchvision.transforms.ColorJitter(contrast=0.4, brightness=0.3),
    # Should randomly add black boxes, but doesn't seem to work
    torchvision.transforms.RandomErasing(p=0.3, scale=(0.01, 0.03), ratio=(1 / 3, 3)),
    # Convert back to torch.tensor
    torchvision.transforms.ToTensor(),
])

In [85]:
a = np.arange(70)
for maxDepth in range(1, 5):
    #define classifier here
    clf = RandomForestClassifier(max_depth = maxDepth, random_state = 0)
    #initialize true positive/false negative/false positive counts (define f1 score)
    tp = 0
    fn = 0
    fp = 0
    folds =2
    for fold in range(folds):
        Xtrain = X[a%folds != fold, :]
        ytrain = y[a%folds != fold]
        Xval = X[a%folds == fold, :]
        yval = y[a%folds == fold]
        print(ytrain)
        #in order to get augment to work...
        imgs = np.zeros((Xtrain.shape[0],512,512,3)) #will be Xtrain, but I wanted this to run faster
        for idx in range(Xtrain.shape[0]):
            obs = np.reshape(Xtrain[idx],(512,512))
            img = np.zeros((512,512,3))
            for i in range(512):
                for j in range(512):
                    for k in range(3):                        
                        img[i,j,:] = obs[i,j]
            imgs[idx,:,:,:] = img
        Xtrain, ytrain = augment(imgs, ytrain, TRANSFORMATIONS) #
        #Xtrain, ytrain = augment(Xtrain, ytrain, TRANSFORMATIONS)
        clf.fit(Xtrain, ytrain)
        if folds == 70:
            preds = clf.predict(Xtest[np.newaxis(), :]) #np.newaxis might be unique to folds 70
        else:
            preds = clf.predict(Xtest)
        #print(str(pred[0]) + ","+str(y[holdout]))
        for i in range (len(pred)):
            if (pred[i] == 1 and yval[i] == 1):
                tp += 1
            else:
                if((pred[i] == 1) and (yval[i] == 0)):
                    fp += 1
                else:
                    if((pred[i] == 0) and (yval[i] == 1)):
                        fn += 1
    p = tp/(tp+fn)
    r = tp/(tp+fp)
    f1 = 2*p*r/(p+r)
    print("MaxDepth: "+ str(maxDepth) + ", errorrate = "+ str((fp+fn)/70) + ", f1 = " + str(f1) + ", p = " + str(p) + ", r = " + str(r))

[1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
4718592
(6, 512, 512, 3)
[[[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.2274482  0.2274482  0.2274482 ]
  [0.22352681 0.22352681 0.22352681]
  [0.21568403 0.21568403 0.21568403]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.23136958 0.23136958 0.23136958]
  [0.23921236 0.23921236 0.23921236]
  [0.2274482  0.2274482  0.2274482 ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.24705514 0.24705514 0.24705514]
  [0.25098413 0.25098413 0.25098413]
  [0.24313375 0.24313375 0.24313375]]

 ...

 [[0.99999994 0.99999994 0.99999994]
  [0.99999994 0.99999994 0.99999994]
  [0.99999994 0.99999994 0.99999994]
  ...
  [0.99215716 0.99215716 0.99215716]
  [0.98823577 0.9

NameError: name 'verbose_print' is not defined

In [9]:
from sklearn.ensemble import RandomForestClassifier

X = shuffled_X
y = shuffled_y

a = np.arange(70)
for maxDepth in range(1, 5):
    tp = 0
    fn = 0
    fp = 0
    clf = convolutionForests(maxDepth = maxDepth)
    #clf = RandomForestClassifier(max_depth = maxDepth, random_state=0)
    folds = 70
    for fold in range(folds):
        clf.fit(X[a%folds != fold, :], y[a%folds != fold])
        pred = clf.predict(X[np.newaxis, a[a%folds == fold], :]) #np.newaxis might be unique to folds 70
        #print(str(pred[0]) + ","+str(y[holdout]))
        if (pred[0] == 1 and y[holdout] == 1):
            tp += 1 
        else:
            if((pred[0] == 1) and (y[holdout] == 0)):
                fp += 1
            else:
                if((pred[0] == 0) and (y[holdout] == 1)):
                    fn += 1
    p = tp/(tp+fn)
    r = tp/(tp+fp)
    f1 = 2*p*r/(p+r)
    print("MaxDepth: "+ str(maxDepth) + ", errorrate = "+ str((fp+fn)/70) + ", f1 = " + str(f1) + ", p = " + str(p) + ", r = " + str(r))

NameError: name 'convolutionForests' is not defined

In [29]:
from sklearn.ensemble import RandomForestClassifier

class convolutionForests:
    
    def __init__(self, maxDepth = 3):
        self.model = RandomForestClassifier(max_depth = maxDepth, random_state=0)
    def fit(self,X,y):
        self.convolve(X,y)
        self.model.fit(X,y)
    
    def convolve(self,X,y):
        #shifts:
        for shift in np.arange(1,4):
            Xshifted = np.zeros(X.shape)
            for row in range(X.shape[0]):
                Xshifted[row, shift:X.shape[1]] = X[row, 0:-1*shift]
            X = np.concatenate((Xshifted, X), axis = 0)
            y = np.concatenate((y,y), axis=0)
        #print(y)
    def predict(self, Xtest):
        return self.model.predict(Xtest)
        